In [0]:
from pyspark.sql.functions import current_timestamp, col

In [0]:
# input_path = "s3://thotadhanasekhar0001/CUSTOMERS/"

input_path = "/Volumes/my_catalog/raw_retailx/customers/*/" 
bronze_tbl = "my_catalog.bronze_retailx.customers"
checkpoint_base = "/Volumes/my_catalog/_system/checkpoint/retailx_customers/"

schema_path = checkpoint_base + "schema"
checkpoint_path = checkpoint_base + "checkpoint"

In [0]:
df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
    .option("cloudFiles.schemaLocation", schema_path)
    .load(input_path)
)
# .option("pathGlobaFilter", "*.csv")
# .option("cloudFiles.maxFilesPerTrigger", "1")
#.option("cloudFiles.maxBytesPerTrigger", "100000000")

In [0]:
df_bronze = (
    df
    .withColumn("_ingested_at", current_timestamp())
    .withColumn("_source_file", col("_metadata.file_path"))
    #.withColumn("_file", col("_metadata.file_name"))
    #.withColumn("_file_mod_time", col("_metadata.file_modification_time"))
)

In [0]:
(
    df_bronze.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path) 
    .trigger(availableNow=True) 
    .toTable(bronze_tbl)     
)

In [0]:
df_customers = spark.read.table(bronze_tbl)
display(df_customers)

CUSTOMER_ID,NAME,EMAIL,CREATED_DATE,_rescued_data,_ingested_at,_source_file
1,Ravi Kumar,ravi.kumar@gmail.com,2025-01-05,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
2,Sneha Reddy,sneha.reddy@gmail.com,2025-01-10,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
3,Arjun Mehta,arjun.mehta@gmail.com,2025-01-15,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
4,Priya Sharma,priya.sharma@gmail.com,2025-02-01,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
5,Vikram Singh,vikram.singh@gmail.com,2025-02-10,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
null,Ravi Kumar,ravi.kumar@gmail.com,2025-01-05,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
12,null,sneha.reddy@gmail.com,2025-01-10,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
13,Arjun Mehta,arjun.mehtagmail.com,2025-01-15,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
4,Priya Sharma,priya.sharma@gmail.com,2025-02-01,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
5,Vikram Singh,vikram.singh@gmail.com,2025-02-10,null,2025-12-14T05:37:11.351Z,/Volumes/my_catalog/raw_retailx/customers/2025/12/14/Customers%201.csv
